In [1]:
import torch
import clip
import pandas as pd
from PIL import Image
import os

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import clip
import pandas as pd
from PIL import Image
import os

# CLIP 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 이미지 디렉토리 설정
image_folder = "./own_dataset_frames"  # 이미지가 저장된 폴더
image_filenames = [
    os.path.join(image_folder, fname)
    for fname in os.listdir(image_folder)
    if fname.endswith(('.jpg', '.png', '.jpeg'))
]

# 이미지 특징 추출
images = []
image_features = []

print("이미지 특징을 추출 중입니다...")
for image_path in image_filenames:
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    images.append(image)

with torch.no_grad():
    image_features = torch.cat([model.encode_image(image) for image in images])
    image_features /= image_features.norm(dim=-1, keepdim=True)  # 정규화

print("이미지 특징 추출 완료!")

# 쿼리 CSV 파일 로드
query_csv_path = "own_dataset_v2.csv"  # 입력 CSV 파일 경로
queries = pd.read_csv(query_csv_path)

# 결과 저장용 리스트
results = []

# 각 쿼리에 대해 가장 유사한 이미지(top-1) 찾기
for idx, row in queries.iterrows():
    query_text = row['query']

    # 텍스트 특징 추출
    text_tokens = clip.tokenize([query_text]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens)
        text_features /= text_features.norm(dim=-1, keepdim=True)  # 정규화

    # 유사도 계산
    similarity = (100.0 * image_features @ text_features.T).squeeze(1)
    top_index = similarity.topk(1).indices.cpu().numpy()[0]

    # 이미지 파일 이름에서 video_id와 time 추출
    matched_image = os.path.basename(image_filenames[top_index])
    video_id = matched_image.split('_')[0]
    time = int(matched_image.split('_')[-1].split('.')[0])
    start_end = time / 2

    # 결과 추가
    results.append({
        "query": query_text,
        "video_id": video_id,
        "start": start_end,
        "end": start_end
    })

# 결과를 CSV 파일로 저장
output_csv_path = ".retrieved_results.csv"  # 출력 CSV 파일 경로
results_df = pd.DataFrame(results)
results_df.to_csv(output_csv_path, index=False)

print(f"결과가 {output_csv_path}에 저장되었습니다.")

이미지 특징을 추출 중입니다...
이미지 특징 추출 완료!
결과가 .retrieved_results.csv에 저장되었습니다.


In [3]:
import torch
import clip
import pandas as pd
from PIL import Image
import os

# CLIP 모델 로드
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 이미지 디렉토리 설정
image_folder = "./own_dataset_frames"  # 이미지가 저장된 폴더
image_filenames = [
    os.path.join(image_folder, fname)
    for fname in os.listdir(image_folder)
    if fname.endswith(('.jpg', '.png', '.jpeg'))
]

# 이미지 특징 추출
images = []
image_features = []

print("이미지 특징을 추출 중입니다...")
for image_path in image_filenames:
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    images.append(image)

with torch.no_grad():
    image_features = torch.cat([model.encode_image(image) for image in images])
    image_features /= image_features.norm(dim=-1, keepdim=True)  # 정규화

print("이미지 특징 추출 완료!")

# 쿼리 CSV 파일 로드
query_csv_path = "own_dataset_v2.csv"  # 입력 CSV 파일 경로
queries = pd.read_csv(query_csv_path)

# 결과 저장용 리스트
results = []

# 각 쿼리에 대해 상위 5개 이미지를 찾기
top_k = 5  # 추출할 상위 이미지 개수
for idx, row in queries.iterrows():
    query_text = row['query']

    # 텍스트 특징 추출
    text_tokens = clip.tokenize([query_text]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens)
        text_features /= text_features.norm(dim=-1, keepdim=True)  # 정규화

    # 유사도 계산
    similarity = (100.0 * image_features @ text_features.T).squeeze(1)
    top_indices = similarity.topk(top_k).indices.cpu().numpy()

    # 상위 5개의 결과를 처리
    for rank, top_index in enumerate(top_indices, 1):
        # 이미지 파일 이름에서 video_id와 time 추출
        matched_image = os.path.basename(image_filenames[top_index])
        video_id = matched_image.split('_')[0]
        time = int(matched_image.split('_')[-1].split('.')[0])
        start_end = time / 2

        # 결과 추가
        results.append({
            "original_query": query_text,
            "video_id": video_id,
            "start": start_end,
            "end": start_end
        })

# 결과를 CSV 파일로 저장
output_csv_path = "./retrieved_results_top5.csv"  # 출력 CSV 파일 경로
results_df = pd.DataFrame(results)
results_df.to_csv(output_csv_path, index=False)

print(f"결과가 {output_csv_path}에 저장되었습니다.")


이미지 특징을 추출 중입니다...
이미지 특징 추출 완료!
결과가 ./retrieved_results_top5.csv에 저장되었습니다.
